In [24]:
#Imports & DB path
import sqlite3
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import traceback

db_path = "student_recommendation.db"

In [25]:
#Create database and tables (run once)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS study_logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_name TEXT,
    subject TEXT,
    hours INTEGER,
    topics_covered TEXT,
    date TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS admin_users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT UNIQUE,
    password TEXT
)
""")

# Insert default admin if not exists
cursor.execute("INSERT OR IGNORE INTO admin_users (username, password) VALUES (?, ?)", ("admin", "admin123"))

conn.commit()
conn.close()
print("Database and tables ready.")

Database and tables ready.


In [26]:
# Cell 3: Student interface widgets (define before functions)
student_name = widgets.Text(description="Name:", placeholder="Enter student name")
subject = widgets.Text(description="Subject:", placeholder="Enter subject")
hours = widgets.IntSlider(description="Hours", min=1, max=12, value=1)
topics = widgets.Textarea(description="Topics:", placeholder="Topics covered (comma separated)", layout=widgets.Layout(width='60%'))
date = widgets.DatePicker(description="Date")

submit_btn = widgets.Button(description="Save Log", button_style="success")
show_btn = widgets.Button(description="Show All Logs", button_style="info")

output = widgets.Output()
logs_output = widgets.Output()

# Display UI (buttons not yet connected)
display(student_name, subject, hours, topics, date, submit_btn, show_btn, output, logs_output)

Text(value='', description='Name:', placeholder='Enter student name')

Text(value='', description='Subject:', placeholder='Enter subject')

IntSlider(value=1, description='Hours', max=12, min=1)

Textarea(value='', description='Topics:', layout=Layout(width='60%'), placeholder='Topics covered (comma separ…

DatePicker(value=None, description='Date', step=1)

Button(button_style='success', description='Save Log', style=ButtonStyle())

Button(button_style='info', description='Show All Logs', style=ButtonStyle())

Output()

Output()

In [27]:
# Cell 4: save_log function (robust, resets fields, prints exceptions)
def save_log(b):
    with output:
        clear_output()
        # Validation
        if not student_name.value.strip():
            print("Please enter student name.")
            return
        if not subject.value.strip():
            print("Please enter subject.")
            return
        if date.value is None:
            print("Please pick a date.")
            return

        try:
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()

            cursor.execute("""
                INSERT INTO study_logs
                (student_name, subject, hours, topics_covered, date)
                VALUES (?, ?, ?, ?, ?)
            """, (
                student_name.value.strip(),
                subject.value.strip(),
                int(hours.value),
                topics.value.strip(),
                str(date.value)
            ))
            conn.commit()

            print("✔ Study log saved successfully!")

            # Reset fields for next entry
            student_name.value = ""
            subject.value = ""
            hours.value = 1
            topics.value = ""
            date.value = None

        except Exception as e:
            traceback.print_exc()
            print("Error:", e)
        finally:
            try:
                conn.close()
            except:
                pass

In [14]:
#show logs function and attach save handler
def show_logs(b=None):
    with logs_output:
        clear_output()
        try:
            conn = sqlite3.connect(db_path)
            df = pd.read_sql_query("SELECT * FROM study_logs ORDER BY id DESC", conn)
            conn.close()
            display(df)
        except Exception as e:
            traceback.print_exc()
            print("Error:", e)

# Attach callbacks
submit_btn.on_click(save_log)
show_btn.on_click(show_logs)

# Optional: show initial table
show_logs()

In [28]:
#Admin login widgets (define before admin functions)
admin_user = widgets.Text(description="Admin User:")
admin_pass = widgets.Password(description="Password:")
login_btn = widgets.Button(description="Login", button_style="primary")
admin_output = widgets.Output()

display(admin_user, admin_pass, login_btn, admin_output)

Text(value='', description='Admin User:')

Password(description='Password:')

Button(button_style='primary', description='Login', style=ButtonStyle())

Output()

In [30]:
# Cell 7: Admin login and admin panel UI builder
def admin_panel_UI():
    # Admin CRUD widgets
    show_btn = widgets.Button(description="Show Logs", button_style="info")
    delete_id = widgets.IntText(description="Delete ID:")
    delete_btn = widgets.Button(description="Delete", button_style="danger")
    update_id = widgets.IntText(description="Update ID:")
    new_hours = widgets.IntText(description="New Hours:")
    update_btn = widgets.Button(description="Update", button_style="warning")
    create_student_name = widgets.Text(description="Name:")
    create_subject = widgets.Text(description="Subject:")
    create_hours = widgets.IntText(description="Hours", value=1)
    create_topics = widgets.Text(description="Topics:")
    create_date = widgets.DatePicker(description="Date")
    create_btn = widgets.Button(description="Create Log", button_style="success")
    panel_output = widgets.Output()

    # Actions
    def show_data(b):
        with panel_output:
            clear_output()
            show_logs()

    def delete_row(b):
        with panel_output:
            clear_output()
            try:
                conn = sqlite3.connect(db_path)
                cursor = conn.cursor()
                cursor.execute("DELETE FROM study_logs WHERE id=?", (delete_id.value,))
                conn.commit()
                conn.close()
                print("✔ Record Deleted (if existed)")
                show_logs()
            except Exception as e:
                traceback.print_exc()
                print("Error:", e)

    def update_row(b):
        with panel_output:
            clear_output()
            try:
                conn = sqlite3.connect(db_path)
                cursor = conn.cursor()
                cursor.execute("UPDATE study_logs SET hours=? WHERE id=?", (new_hours.value, update_id.value))
                conn.commit()
                conn.close()
                print("Record Updated (if id existed)")
                show_logs()
            except Exception as e:
                traceback.print_exc()
                print("Error:", e)

    def create_row(b):
        with panel_output:
            clear_output()
            if not create_student_name.value.strip():
                print("Provide name")
                return
            if not create_subject.value.strip():
                print("Provide subject")
                return
            if create_date.value is None:
                print("Provide date")
                return
            try:
                conn = sqlite3.connect(db_path)
                cursor = conn.cursor()
                cursor.execute("""
                    INSERT INTO study_logs (student_name, subject, hours, topics_covered, date)
                    VALUES (?, ?, ?, ?, ?)
                """, (
                    create_student_name.value.strip(),
                    create_subject.value.strip(),
                    int(create_hours.value),
                    create_topics.value.strip(),
                    str(create_date.value)
                ))
                conn.commit()
                conn.close()
                print("✔ Created log")
                show_logs()
            except Exception as e:
                traceback.print_exc()
                print("Error:", e)

    # Attach handlers
    show_btn.on_click(show_data)
    delete_btn.on_click(delete_row)
    update_btn.on_click(update_row)
    create_btn.on_click(create_row)

    # Layout
    row1 = widgets.HBox([show_btn])
    row2 = widgets.HBox([delete_id, delete_btn])
    row3 = widgets.HBox([update_id, new_hours, update_btn])
    create_box = widgets.VBox([
        widgets.HTML("<b>Create New Study Log (Admin)</b>"),
        widgets.HBox([create_student_name, create_subject]),
        widgets.HBox([create_hours, create_topics, create_date]),
        create_btn
    ])

    return widgets.VBox([row1, row2, row3, create_box, panel_output])

def admin_login(b):
    with admin_output:
        clear_output()

        # Simple hardcoded login
        if admin_user.value == "admin" and admin_pass.value == "admin123":
            print("✔ Login Successful\n")
            display(admin_panel_UI())
        else:
            print("Invalid Credentials")
login_btn.on_click(admin_login)